## Performance of Triton Inference Server

In [ ]:
!pip install bash_kernel tritonclient

In [6]:
# runs inside Jupyter container
!perf_analyzer -u 129.114.109.59:8010 -m caption --input-data input.json -b 1

 Successfully read data for 1 stream/streams with 1 step/steps.
*** Measurement Settings ***
  Batch size: 1
  Service Kind: TRITON
  Using "time_windows" mode for stabilization
  Stabilizing using average latency and throughput
  Measurement window: 5000 msec
  Using synchronous calls for inference

Request concurrency: 1
  Client: 
    Request count: 76
    Throughput: 4.21707 infer/sec
    Avg latency: 236794 usec (standard deviation 3284 usec)
    p50 latency: 236005 usec
    p90 latency: 239818 usec
    p95 latency: 240915 usec
    p99 latency: 249794 usec
    Avg HTTP time: 236778 usec (send/recv 591 usec + response wait 236187 usec)
  Server: 
    Inference count: 76
    Execution count: 76
    Successful request count: 76
    Avg request latency: 234123 usec (overhead 4 usec + queue 57 usec + compute input 93 usec + compute infer 233901 usec + compute output 67 usec)
Inferences/Second vs. Client Average Batch Latency
Concurrency: 1, throughput: 4.21707 infer/sec, latency 236794

<!--

    Avg request latency: 18689 usec (overhead 2 usec + queue 22 usec + compute input 44 usec + compute infer 18570 usec + compute output 49 usec)

Inferences/Second vs. Client Average Batch Latency
Concurrency: 1, throughput: 51.549 infer/sec, latency 19311 usec

-->

Let’s further exercise this service. In the command above, a single client sends continuous requests to the server - each time a response is returned, a new request is generated. Now, let’s configure **8** concurrent clients, each sending continuous requests - as soon as any client gets a response, it sends a new request:

In [9]:
# runs inside Jupyter container
!perf_analyzer -u 129.114.109.59:8010  -m caption  --input-data input.json -b 1 --concurrency-range 8

 Successfully read data for 1 stream/streams with 1 step/steps.
*** Measurement Settings ***
  Batch size: 1
  Service Kind: TRITON
  Using "time_windows" mode for stabilization
  Stabilizing using average latency and throughput
  Measurement window: 5000 msec
  Using synchronous calls for inference

Request concurrency: 8
  Client: 
    Request count: 79
    Throughput: 4.38366 infer/sec
    Avg latency: 1843192 usec (standard deviation 5291 usec)
    p50 latency: 1841055 usec
    p90 latency: 1850511 usec
    p95 latency: 1851137 usec
    p99 latency: 1851592 usec
    Avg HTTP time: 1843172 usec (send/recv 375 usec + response wait 1842797 usec)
  Server: 
    Inference count: 79
    Execution count: 79
    Successful request count: 79
    Avg request latency: 1840301 usec (overhead 2 usec + queue 1609898 usec + compute input 78 usec + compute infer 230250 usec + compute output 72 usec)
Inferences/Second vs. Client Average Batch Latency
Concurrency: 8, throughput: 4.38366 infer/sec, l

<!-- 

    Avg request latency: 151375 usec (overhead 3 usec + queue 132341 usec + compute input 59 usec + compute infer 18922 usec + compute output 49 usec)

Inferences/Second vs. Client Average Batch Latency
Concurrency: 8, throughput: 52.3786 infer/sec, latency 151983 usec

-->

While the inference time (`compute infer`) remains low, the overall system latency is high because of `queue` delay. Only one sample is processed at a time, and other samples have to wait in a queue for their turn. Here, since there are 8 concurrent clients sending continuous requests, the delay is approximately 8x the inference delay. With more concurrent requests, the queuing delay would grow even larger:

In [10]:
# runs inside Jupyter container
!perf_analyzer -u 129.114.109.59:8010  -m caption  --input-data input.json -b 1 --concurrency-range 16

 Successfully read data for 1 stream/streams with 1 step/steps.
*** Measurement Settings ***
  Batch size: 1
  Service Kind: TRITON
  Using "time_windows" mode for stabilization
  Stabilizing using average latency and throughput
  Measurement window: 5000 msec
  Using synchronous calls for inference

Request concurrency: 16
  Client: 
    Request count: 78
    Throughput: 4.32175 infer/sec
    Avg latency: 3712014 usec (standard deviation 23192 usec)
    p50 latency: 3730049 usec
    p90 latency: 3734920 usec
    p95 latency: 3736303 usec
    p99 latency: 3738495 usec
    Avg HTTP time: 3712328 usec (send/recv 416 usec + response wait 3711912 usec)
  Server: 
    Inference count: 77
    Execution count: 77
    Successful request count: 77
    Avg request latency: 3709336 usec (overhead 3 usec + queue 3477082 usec + compute input 83 usec + compute infer 232098 usec + compute output 70 usec)
Inferences/Second vs. Client Average Batch Latency
Concurrency: 16, throughput: 4.32175 infer/sec

<!-- 

    Avg request latency: 302079 usec (overhead 1 usec + queue 283040 usec + compute input 60 usec + compute infer 18927 usec + compute output 50 usec)

Inferences/Second vs. Client Average Batch Latency
Concurrency: 16, throughput: 52.3609 infer/sec, latency 302804 usec

-->

Although the delay is large (over 100 ms), it’s not because of inadequate compute - if you check the `nvtop` display on the host while the test above is running, you will note low GPU utilization! Take a screenshot of the `nvtop` output when this test is running.

We *could* get more throughput without increasing prediction latency, by batching requests:

In [11]:
# runs inside Jupyter container
!perf_analyzer -u 129.114.109.59:8010  -m caption  --input-data input.json -b 16 --concurrency-range 1

 Successfully read data for 1 stream/streams with 1 step/steps.
*** Measurement Settings ***
  Batch size: 16
  Service Kind: TRITON
  Using "time_windows" mode for stabilization
  Stabilizing using average latency and throughput
  Measurement window: 5000 msec
  Using synchronous calls for inference

Request concurrency: 1
  Client: 
    Request count: 72
    Throughput: 63.9023 infer/sec
    Avg latency: 248505 usec (standard deviation 2625 usec)
    p50 latency: 248115 usec
    p90 latency: 251286 usec
    p95 latency: 253176 usec
    p99 latency: 257178 usec
    Avg HTTP time: 248489 usec (send/recv 4790 usec + response wait 243699 usec)
  Server: 
    Inference count: 1152
    Execution count: 72
    Successful request count: 72
    Avg request latency: 235269 usec (overhead 3 usec + queue 65 usec + compute input 731 usec + compute infer 234406 usec + compute output 64 usec)
Inferences/Second vs. Client Average Batch Latency
Concurrency: 1, throughput: 63.9023 infer/sec, latency 2

<!--

    Avg request latency: 21189 usec (overhead 3 usec + queue 19 usec + compute input 195 usec + compute infer 20921 usec + compute output 50 usec)

Inferences/Second vs. Client Average Batch Latency
Concurrency: 1, throughput: 656.63 infer/sec, latency 24282 usec


-->

But, that’s not very helpful in a situation when requests come from individual users, one at a time.

<!--

    Avg request latency: 100423 usec (overhead 6 usec + queue 44892 usec + compute input 197 usec + compute infer 55111 usec + compute output 216 usec)

Inferences/Second vs. Client Average Batch Latency
Concurrency: 8, throughput: 78.6276 infer/sec, latency 101232 usec

-->